In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [2]:
%run "{project_path}\llm_custom_apps\llm_apps\oh_sheet_its_spark\utils.ipynb"

In [4]:
import gradio as gr

# Initialize an empty list to maintain chat history & Message history
chat_history = []
msg_history = []
inf_mdl_nm = "Qwen/Qwen2.5-Coder-32B-Instruct"
file_uploaded = False  # Flag to track if a file is uploaded

# Function to handle file upload
def handle_file_upload(file):
    global file_uploaded, chat_history, msg_history
    if file:
        # Reset histories on new file upload
        chat_history = []
        msg_history = []

        file_uploaded = True  # Set the flag to True when a file is uploaded

        # Extract the file name
        file_name = file.name

        initial_msg = gen_initial_msg(file_name)
        cd_resp = generate_responses_from_inf(hf_token, initial_msg, inf_mdl_nm, 5000)

        # Add the "File Loaded" message to msg_history only
        msg_history.append(initial_msg[0])
        msg_history.append(initial_msg[1])
        
        # Add the bot's response with the file name to both chat_history and msg_history
        bot_message = f"The file name is: {file_name}"
        chat_history.append(("Bot", "\n"+cd_resp))
        msg_history.append({'role': "assistant", 'content': cd_resp})

        # Generate the formatted chat history
        conversation_history = format_chat_history(chat_history)

        return "File uploaded successfully. You can now start chatting!", conversation_history
    else:
        file_uploaded = False
        return "Please upload a valid Excel file.", ""

# Function to format chat history for display
def format_chat_history(chat_history):
    conversation_history = ''
    for speaker, message in chat_history:
        if speaker == "User":
            conversation_history += f'**{speaker}:** {message}\n\n'
        else:
            conversation_history += f'**{speaker}:** {message}\n\n'
    return conversation_history

# Function to enable chat input and submit button after file upload
def enable_chat_components(file_status):
    if "successfully" in file_status.lower():
        return gr.update(interactive=True), gr.update(interactive=True)
    else:
        return gr.update(interactive=False), gr.update(interactive=False)

# Function to handle chat history and generate responses
def chatbot_history(user_input):
    global chat_history  # Use the global chat_history list
    global msg_history

    # Append user input to the chat history
    chat_history.append(("User", user_input))
    msg_history.append({'role': "user", 'content': user_input})

    # Generate a response (use updated message history for the assistant)
    cd_resp = generate_responses_from_inf(hf_token, msg_history, inf_mdl_nm, 5000)
    #cd_resp = user_input[::-1]  # For testing, reversing user input as the bot's response
    msg_history.append({'role': "assistant", 'content': cd_resp})

    # Append bot response to chat history
    chat_history.append(("Bot", "\n" + cd_resp))

    # Generate the full chat history for display, not just the latest message
    conversation_history = format_chat_history(chat_history)

    # Return the formatted conversation history and clear the input field
    return gr.update(value=conversation_history), ""  # Update the full conversation history

# Create Gradio interface using Blocks with Soft theme
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # Add a title with proper font
    gr.HTML(""" 
        <div style="font-size: 2.5em; color: #4CAF50; font-weight: bold; text-align: center;">
            Oh Sheet!!!!! It's Spark
        </div>
    """)

    # Layout with chat and input section
    with gr.Row():
        # Message section on the left (30% width)
        with gr.Column(scale=1, min_width=30):
            # File uploader at the top
            file_upload = gr.File(label="Upload an Excel File", file_types=[".xls", ".xlsx"], interactive=True)
            file_status = gr.Textbox(label="", interactive=False, lines=1, value="Please upload an Excel file to begin.", show_label=False)

            # Chatbox and submit button
            input_text = gr.Textbox(label="Enter your message", lines=1, interactive=False)  # Initially disabled
            submit_button = gr.Button("Submit", variant="primary", interactive=False)  # Initially disabled

        # Chatbot section on the right (70% width)
        with gr.Column(scale=2, min_width=70):
            output_text = gr.Markdown(label="Chatbot History", elem_id="chat_history")  # Use Markdown for rendering content

    # Ensure file upload enables the chatbox
    file_upload.change(fn=handle_file_upload, inputs=file_upload, outputs=[file_status, output_text])
    file_status.change(fn=enable_chat_components, inputs=file_status, outputs=[input_text, submit_button])

    # Ensure Enter key is mapped to submit functionality
    input_text.submit(fn=chatbot_history, inputs=input_text, outputs=[output_text, input_text])
    submit_button.click(fn=chatbot_history, inputs=input_text, outputs=[output_text, input_text])

# Launch the app
demo.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
